In [1]:
import pandas as pd
data = pd.read_csv('data/housing.csv')
data.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


In [2]:
X = data.copy().drop(["ocean_proximity"], axis=1)
Y = data.copy()["ocean_proximity"]
Y = data.copy()["ocean_proximity"].map({"<1H OCEAN":0, "INLAND":1, "ISLAND": 2, "NEAR BAY": 3, "NEAR OCEAN": 4}).values

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, train_size=0.8, random_state=1)
X_train, X_dev, y_train, y_dev = train_test_split(X_train, y_train, test_size=0.2, train_size=0.8, random_state=1)

In [4]:
from sklearn.preprocessing import Imputer 
from sklearn import preprocessing

imputer = Imputer(strategy="median")
imputer.fit(X_train)

X_train = imputer.transform(X_train)
X_dev = imputer.transform(X_dev)
X_test = imputer.transform(X_test)

scaler = preprocessing.StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_dev = scaler.transform(X_dev)
X_test = scaler.transform(X_test)

/home/joe/anaconda3/envs/tensorflow/lib/python3.6/site-packages/sklearn/utils/deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [5]:
# Input Features:
print(X_train.shape)
print(X_dev.shape)
print(X_test.shape)
print(X_train[:3])

(13209, 9)
(3303, 9)
(4128, 9)
[[ 0.89872022 -0.89773476 -1.24376976 -0.32171094 -0.69583159 -0.43931447
  -0.67219421  2.24405982  1.48686249]
 [ 0.68448647 -0.84637731 -0.60683118 -0.17791607 -0.20472295 -0.45260971
  -0.20959953  0.38636939  0.55799804]
 [ 0.92363112 -0.98177423 -1.16415244 -0.44476616 -0.5808403  -0.63785671
  -0.54340365  0.65584018  0.37757734]]


In [6]:
# Labels:
print(y_train.shape)
print(y_dev.shape)
print(y_test.shape)
print(y_train[:10])

(13209,)
(3303,)
(4128,)
[0 4 0 1 0 0 1 0 4 4]


## Model 1:

In [7]:
def get_accuracy(predictions,target):
    count_correct = 0
    total = len(predictions)
    for i in range(len(predictions)):
        p = predictions[i]
        t = target[i]
        if p == t: 
            count_correct += 1
    accuracy = round(count_correct/total,7)*100
    return accuracy

In [8]:
import tensorflow as tf
import time

In [9]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 5
lr = 1.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 5
Learning Rate: 1.0
Total Epochs: 50
Accuracy after epoch 0 = 18.76751%
Accuracy after epoch 10 = 68.21106999999999%
Accuracy after epoch 20 = 73.26823%
Accuracy after epoch 30 = 75.95579000000001%
Accuracy after epoch 40 = 77.13680000000001%
Time taken = 1.43301 seconds


In [10]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 5
lr = 0.1
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 5
Learning Rate: 0.1
Total Epochs: 50
Accuracy after epoch 0 = 26.73934%
Accuracy after epoch 10 = 47.31622%
Accuracy after epoch 20 = 53.81937%
Accuracy after epoch 30 = 56.84003%
Accuracy after epoch 40 = 59.04308%
Time taken = 0.68165 seconds


In [11]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 5
lr = 0.1
epochs = 100

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 5
Learning Rate: 0.1
Total Epochs: 100
Accuracy after epoch 0 = 22.41653%
Accuracy after epoch 10 = 45.3403%
Accuracy after epoch 20 = 57.64252%
Accuracy after epoch 30 = 61.140130000000006%
Accuracy after epoch 40 = 62.48013%
Accuracy after epoch 50 = 63.32046%
Accuracy after epoch 60 = 64.13809%
Accuracy after epoch 70 = 65.16012%
Accuracy after epoch 80 = 66.48497%
Accuracy after epoch 90 = 68.12779%
Time taken = 1.93598 seconds


In [12]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 1.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 10
Learning Rate: 1.0
Total Epochs: 50
Accuracy after epoch 0 = 15.080630000000001%
Accuracy after epoch 10 = 69.52078%
Accuracy after epoch 20 = 74.03286%
Accuracy after epoch 30 = 76.58415%
Accuracy after epoch 40 = 77.67431%
Time taken = 1.8655 seconds


In [13]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 1.0
epochs = 100

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 10
Learning Rate: 1.0
Total Epochs: 100
Accuracy after epoch 0 = 20.44061%
Accuracy after epoch 10 = 68.49875%
Accuracy after epoch 20 = 73.42721%
Accuracy after epoch 30 = 75.97093000000001%
Accuracy after epoch 40 = 77.22008%
Accuracy after epoch 50 = 77.90899999999999%
Accuracy after epoch 60 = 78.37081%
Accuracy after epoch 70 = 78.71148%
Accuracy after epoch 80 = 78.88561%
Accuracy after epoch 90 = 79.27928%
Time taken = 3.59455 seconds


In [14]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 1.0
epochs = 100

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 10
Learning Rate: 1.0
Total Epochs: 100
Accuracy after epoch 0 = 17.50322%
Accuracy after epoch 10 = 69.22553%
Accuracy after epoch 20 = 74.82777%
Accuracy after epoch 30 = 77.07624%
Accuracy after epoch 40 = 77.99985000000001%
Accuracy after epoch 50 = 78.68876999999999%
Accuracy after epoch 60 = 79.18086%
Accuracy after epoch 70 = 79.86979%
Accuracy after epoch 80 = 80.6647%
Accuracy after epoch 90 = 79.10516%
Accuracy on Dev Set: 0.8044202327728271
Time taken = 1.53487 seconds


In [15]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 50
lr = 1.0
epochs = 100

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 50
Learning Rate: 1.0
Total Epochs: 100
Accuracy after epoch 0 = 27.6781%
Accuracy after epoch 10 = 73.67704%
Accuracy after epoch 20 = 76.56901%
Accuracy after epoch 30 = 77.75759%
Accuracy after epoch 40 = 78.68119999999999%
Accuracy after epoch 50 = 79.33227000000001%
Accuracy after epoch 60 = 79.66538%
Accuracy after epoch 70 = 79.92278%
Accuracy after epoch 80 = 78.07555%
Accuracy after epoch 90 = 82.35293999999999%
Accuracy on Dev Set: 0.8234937787055969
Time taken = 3.39026 seconds


In [16]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 5.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Hidden Layer Size: 10
Learning Rate: 5.0
Total Epochs: 50
Accuracy after epoch 0 = 25.21008%
Accuracy after epoch 10 = 62.96465%
Accuracy after epoch 20 = 66.50011%
Accuracy after epoch 30 = 65.86418%
Accuracy after epoch 40 = 73.0941%
Accuracy on Dev Set: 0.7459884881973267
Time taken = 2.57706 seconds


In [17]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 1.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdadeltaOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdadeltaOptimizer
Hidden Layer Size: 10
Learning Rate: 1.0
Total Epochs: 50
Accuracy after epoch 0 = 16.27678%
Accuracy after epoch 10 = 17.934739999999998%
Accuracy after epoch 20 = 19.63813%
Accuracy after epoch 30 = 21.53835%
Accuracy after epoch 40 = 23.12817%
Accuracy on Dev Set: 0.24613986909389496
Time taken = 1.71477 seconds


In [18]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 100.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdadeltaOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdadeltaOptimizer
Hidden Layer Size: 10
Learning Rate: 100.0
Total Epochs: 50
Accuracy after epoch 0 = 15.413730000000001%
Accuracy after epoch 10 = 68.86214000000001%
Accuracy after epoch 20 = 72.10993%
Accuracy after epoch 30 = 73.64676%
Accuracy after epoch 40 = 76.44788%
Accuracy on Dev Set: 0.7677868604660034
Time taken = 0.54959 seconds


In [19]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 20
lr = 200.0
epochs = 200

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdadeltaOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdadeltaOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdadeltaOptimizer
Hidden Layer Size: 20
Learning Rate: 200.0
Total Epochs: 200
Accuracy after epoch 0 = 23.711109999999998%
Accuracy after epoch 10 = 67.80226%
Accuracy after epoch 20 = 69.76304%
Accuracy after epoch 30 = 74.19184%
Accuracy after epoch 40 = 75.19873%
Accuracy after epoch 50 = 77.84844%
Accuracy after epoch 60 = 75.11545%
Accuracy after epoch 70 = 79.74866%
Accuracy after epoch 80 = 77.81815%
Accuracy after epoch 90 = 79.27171%
Accuracy after epoch 100 = 77.27307%
Accuracy after epoch 110 = 78.27239%
Accuracy after epoch 120 = 78.2194%
Accuracy after epoch 130 = 79.52911%
Accuracy after epoch 140 = 78.69633999999999%
Accuracy after epoch 150 = 78.1664%
Accuracy after epoch 160 = 79.4534%
Accuracy after epoch 170 = 80.64956%
Accuracy after epoch 180 = 81.9517%
Accuracy after epoch 190 = 82.26209%
Accuracy on Dev Set: 0.8047229647636414
Time taken = 6.81564 seconds


In [20]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 10
lr = 1.0
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdamOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdamOptimizer
Hidden Layer Size: 10
Learning Rate: 1.0
Total Epochs: 50
Accuracy after epoch 0 = 25.172230000000003%
Accuracy after epoch 10 = 62.84352%
Accuracy after epoch 20 = 62.93436%
Accuracy after epoch 30 = 71.97365%
Accuracy after epoch 40 = 73.98743%
Accuracy on Dev Set: 0.7502270936965942
Time taken = 1.70698 seconds


In [21]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 20
lr = 0.1
epochs = 50

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdamOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdamOptimizer
Hidden Layer Size: 20
Learning Rate: 0.1
Total Epochs: 50
Accuracy after epoch 0 = 16.51147%
Accuracy after epoch 10 = 74.5098%
Accuracy after epoch 20 = 78.31024%
Accuracy after epoch 30 = 81.16436%
Accuracy after epoch 40 = 84.29101%
Accuracy on Dev Set: 0.8504390120506287
Time taken = 0.98015 seconds


In [ ]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 20
lr = 0.1
epochs = 100

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdamOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_dev, target: y_dev, learning_rate: lr})
    print("Accuracy on Dev Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))

Using AdamOptimizer
Hidden Layer Size: 20
Learning Rate: 0.1
Total Epochs: 100
Accuracy after epoch 0 = 17.011129999999998%
Accuracy after epoch 10 = 75.81952%
Accuracy after epoch 20 = 81.17193%
Accuracy after epoch 30 = 83.54152%
Accuracy after epoch 40 = 85.63101%


In [ ]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, [None, 9], name="x")
target = tf.placeholder(tf.int32, [None], name="target")
learning_rate = tf.placeholder(tf.float32, name="learning_rate")

hidden_layer_size = 20
lr = 0.1
epochs = 200

hidden = tf.layers.dense(x, hidden_layer_size, activation=tf.tanh)
output = tf.layers.dense(hidden, 5, activation=None) # changed to 5 because of 5 possible classes

probabilities = tf.nn.softmax(output)
predictions = tf.argmax(probabilities, axis=1)
loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=output, labels=target)
loss = tf.reduce_mean(loss_)
acc, acc_op = tf.metrics.accuracy(target,predictions)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss)

# define info that is used in SummaryWritter
acc_summary = tf.summary.scalar('valid_accuracy', acc)
valid_summary = tf.summary.merge([acc_summary])

data_x = X_train
data_targets = y_train
print("Using AdamOptimizer")
print("Hidden Layer Size: {}".format(hidden_layer_size))
print("Learning Rate: {}".format(lr))
print("Total Epochs: {}".format(epochs))
start_time = time.time()
tf.set_random_seed(20)
with tf.Session() as sess:
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        result, _ = sess.run([predictions, train_op], feed_dict={x: data_x, target: data_targets, learning_rate: lr})
        if epoch % 10 == 0:
            print("Accuracy after epoch {} = {}%".format(epoch,get_accuracy(result,data_targets)))
    
    vsummary = sess.run(acc_op,feed_dict={x: X_test, target: y_test, learning_rate: lr})
    print("Accuracy on Test Set: {}".format(vsummary))
end_time = time.time()
print("Time taken = {} seconds".format(round(end_time-start_time,5)))